In [1]:
from dlt.sources.helpers.rest_client import RESTClient
from dlt.sources.helpers.rest_client.auth import BearerTokenAuth
from dlt.sources.helpers.rest_client.paginators import HeaderLinkPaginator , JSONLinkPaginator

BASE_URL = "https://pokeapi.co/api/v2/pokemon?limit=10"

client = RESTClient(
    base_url= BASE_URL,
    paginator=JSONLinkPaginator(next_url_path="next"),
    data_selector="results",
)

count = 0
max_count = 10

for page in client.paginate():
    for pokemon in page:
        print(pokemon["name"])
        count += 1
        if count > max_count:
            break 
    if count >= max_count:
        break 
    print(page)

bulbasaur
ivysaur
venusaur
charmander
charmeleon
charizard
squirtle
wartortle
blastoise
caterpie


In [2]:
#DLT Resources: 
import dlt 
from dlt.sources.helpers.rest_client import RESTClient
from dlt.sources.helpers.rest_client.auth import BearerTokenAuth
from dlt.sources.helpers.rest_client.paginators import HeaderLinkPaginator , JSONLinkPaginator

BASE_URL = "https://pokeapi.co/api/v2/pokemon?limit=10"

@dlt.resource()
def paginated_gatter():
    client = RESTClient(
        base_url= BASE_URL,
        paginator=JSONLinkPaginator(next_url_path="next"),
        data_selector="results",
    )

    count = 0
    max_count = 10

    for page in client.paginate():
        for pokemon in page:
            print(pokemon["name"])
            count += 1
            if count > max_count:
                break 
        if count >= max_count:
            break 
        yield page 
for record in paginated_gatter():
    print(record)
    

bulbasaur
ivysaur
venusaur
charmander
charmeleon
charizard
squirtle
wartortle
blastoise
caterpie


In [3]:
#dlt config and secrets: 
"""import os
import dlt
from dlt.sources.helpers.rest_client import RESTClient
from dlt.sources.helpers.rest_client.auth import BearerTokenAuth
from dlt.sources.helpers.rest_client.paginators import HeaderLinkPaginator
from google.colab import userdata

os.environ["ACCESS_TOKEN"] = userdata.get('ACCESS_TOKEN') # <--- set ENV in the special format: uppercase secret name
events_data = []


@dlt.resource(name="events")
def paginated_getter(
    access_token=dlt.secrets.value  # <--- set the secret variable "access_token" here
):
    client = RESTClient(
        base_url="https://api.github.com/repos/DataTalksClub/data-engineering-zoomcamp/",
        auth=BearerTokenAuth(token=access_token), # <--- use the variable "access_token" as usual
        paginator=HeaderLinkPaginator(links_next_key="next"),
    )

    for page in client.paginate("events"):
        yield page


for page_data in paginated_getter():
    events_data.append(page_data)
    print(page_data)"""

'import os\nimport dlt\nfrom dlt.sources.helpers.rest_client import RESTClient\nfrom dlt.sources.helpers.rest_client.auth import BearerTokenAuth\nfrom dlt.sources.helpers.rest_client.paginators import HeaderLinkPaginator\nfrom google.colab import userdata\n\nos.environ["ACCESS_TOKEN"] = userdata.get(\'ACCESS_TOKEN\') # <--- set ENV in the special format: uppercase secret name\nevents_data = []\n\n\n@dlt.resource(name="events")\ndef paginated_getter(\n    access_token=dlt.secrets.value  # <--- set the secret variable "access_token" here\n):\n    client = RESTClient(\n        base_url="https://api.github.com/repos/DataTalksClub/data-engineering-zoomcamp/",\n        auth=BearerTokenAuth(token=access_token), # <--- use the variable "access_token" as usual\n        paginator=HeaderLinkPaginator(links_next_key="next"),\n    )\n\n    for page in client.paginate("events"):\n        yield page\n\n\nfor page_data in paginated_getter():\n    events_data.append(page_data)\n    print(page_data)'

In [ ]:
import json
import dlt
import duckdb

# Load JSON from local file
with open("trainers.json") as f:
    data = json.load(f)  # reads the entire JSON array

# Define DLT resource
@dlt.resource(name="trainers", write_disposition="replace")
def load_trainers():
    for row in data:
        yield row

# Define DLT pipeline
pipeline = dlt.pipeline(
    pipeline_name="pokemon_pipeline",
    destination="duckdb",  # or "bigquery", "postgres", etc.
    dataset_name="pokemon"
)

# Run the pipeline
info = pipeline.run(load_trainers)
print(info)


print(pipeline.dataset().schema.data_table_names())


Pipeline pokemon_pipeline load step completed in 1.33 seconds
1 load package(s) were loaded to destination duckdb and into dataset pokemon
The duckdb destination used duckdb:///h:\Data Engineering\Projects\DE Projects\ETL_DLT_pipeline\pokemon_pipeline.duckdb location to store data
Load package 1756006310.096663 is LOADED and contains no failed jobs
['trainers', 'trainers__pokemon', 'trainers__pokemon__type']


In [8]:
#Shows the tables in the db
import duckdb
import numpy as np 
con = duckdb.connect("pokemon_pipeline.duckdb")

# Query the actual table name
result = con.execute("SELECT * FROM pokemon.trainers").fetchdf()
print(result)


   id         name  age location__city location__region       _dlt_load_id  \
0   1  Ash Ketchum   10    Pallet Town            Kanto  1756006310.096663   
1   2        Misty   12  Cerulean City            Kanto  1756006310.096663   

          _dlt_id  
0  6TtNoXJuS7kqaQ  
1  aX3y4rr0CZpCfA  


In [ ]:
#Installs numpy and pandas in the same kernel 
import sys
!"{sys.executable}" -m pip install numpy pandas


   ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
   --------- ------------------------------ 2.9/12.8 MB 15.2 MB/s eta 0:00:01
   --------------------- ------------------ 6.8/12.8 MB 17.5 MB/s eta 0:00:01
   ---------------------------------------  12.6/12.8 MB 21.3 MB/s eta 0:00:01
   ---------------------------------------- 12.8/12.8 MB 19.1 MB/s  0:00:00
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   ---------------------- ----------------- 6.3/11.0 MB 29.6 MB/s eta 0:00:01
   ---------------------------------------  10.7/11.0 MB 29.1 MB/s eta 0:00:01
   ---------------------------------------- 11.0/11.0 MB 25.4 MB/s  0:00:00

   ---------------------------------------- 0/2 [numpy]
   ---------------------------------------- 0/2 [numpy]
   ---------------------------------------- 0/2 [numpy]
   ---------------------------------------- 0/2 [numpy]
   ---------------------------------------- 0/2 [numpy]
   --------------------------------

In [ ]:
#Schema of the table 
schema = con.execute("DESCRIBE pokemon.trainers").fetchdf()
print(schema)


        column_name column_type null   key default extra
0                id      BIGINT  YES  None    None  None
1              name     VARCHAR  YES  None    None  None
2               age      BIGINT  YES  None    None  None
3    location__city     VARCHAR  YES  None    None  None
4  location__region     VARCHAR  YES  None    None  None
5      _dlt_load_id     VARCHAR   NO  None    None  None
6           _dlt_id     VARCHAR   NO  None    None  None


In [28]:
#Flatten the NESTED JSON:
#Sample Nested JSON data:
import pandas as pd

# Example nested JSON with varying numbers of Pokémon and possibly missing fields
data = [
    {
        "name": "Ash",
        "age": 10,
        "location": {"city": "Pallet Town", "region": "Kanto"},
        "pokemon": [
            {"name": "Pikachu", "type": "Electric"},
            {"name": "Bulbasaur", "type": "Grass/Poison"}
        ]
    },
    {
        "name": "Misty",
        "age": 10,
        "location": {"city": "Cerulean City"},  # region missing
        "pokemon": [
            {"name": "Staryu", "type": "Water"},
            {"name": "Psyduck"}  # type missing
        ]
    },
    {
        "name": "Brock",
        "age": 15,
        "location": {"city": "Pewter City", "region": "Kanto"},
        "pokemon": []  # no Pokémon
    }
]

# Flatten the JSON
df = pd.json_normalize(
    data,
    record_path=['pokemon'],  # nested list to explode into rows
    meta=['name', 'age', ['location', 'city'], ['location', 'region']],
    record_prefix="pokemon_",
    errors='ignore'
)
print(df)

  pokemon_name  pokemon_type   name age  location.city location.region
0      Pikachu      Electric    Ash  10    Pallet Town           Kanto
1    Bulbasaur  Grass/Poison    Ash  10    Pallet Town           Kanto
2       Staryu         Water  Misty  10  Cerulean City             NaN
3      Psyduck           NaN  Misty  10  Cerulean City             NaN


In [31]:
#Handling missing values:
df.fillna('', inplace=True)

#Rename columns:
df = df.rename(columns={
    'name': 'pokemon_name',
    'type': 'pokemon_type',
    'name': 'trainer_name',
    'location.city' : 'city',
    'location.region': 'region'
})
print(df)

  pokemon_name  pokemon_type trainer_name age           city region
0      Pikachu      Electric          Ash  10    Pallet Town  Kanto
1    Bulbasaur  Grass/Poison          Ash  10    Pallet Town  Kanto
2       Staryu         Water        Misty  10  Cerulean City       
3      Psyduck                      Misty  10  Cerulean City       


In [ ]:
#Setting up of DATA CONTRACT: 
import dlt
from dlt.sources.helpers.rest_client import RESTClient
from dlt.sources.helpers.rest_client.paginators import JSONLinkPaginator

BASE_URL = "https://pokeapi.co/api/v2/pokemon?limit=10"

@dlt.resource(schema_contract={"tables": "evolve", "columns": "freeze", "data_type": "evolve"})
def paginated_gatter(add_new_col: bool = False, max_count: int = 10):
    client = RESTClient(
        base_url=BASE_URL,
        paginator=JSONLinkPaginator(next_url_path="next"),
        data_selector="results",
    )

    count = 0
    for page in client.paginate():
        for pokemon in page:
            if add_new_col:
                # 🚨 Add new unexpected column
                pokemon["new_col_test"] = "unexpected_value"
            yield pokemon
            count += 1
            if count >= max_count:
                return

pipeline = dlt.pipeline(
    pipeline_name="pokemon_contract",
    destination="duckdb",
    dataset_name="pokemon_dataset"
)

info = pipeline.run(paginated_gatter(add_new_col=True), table_name="pokemons", write_disposition="append")
print(info)


PipelineStepFailed: Pipeline execution failed at `step=normalize` when processing package with `load_id=1756094124.197416` with exception:

<class 'dlt.normalize.exceptions.NormalizeJobFailed'>
Job for `job_id=pokemons.6ba702ac79.typed-jsonl.gz` failed terminally in load with `load_id=1756094124.197416` with message: In schema `pokemon_contract`: In Table: `pokemons` Column: `new_col_test` . Contract on `columns` with `contract_mode=freeze` is violated. Can't add table column `new_col_test` to table `pokemons` because `columns` are frozen. Offending data item: _dlt_id: S26apUUiabzPlQ.

In [ ]:
#Loading with dlt: Using default  method 
import dlt 
from dlt.sources.helpers.rest_client import RESTClient
from dlt.sources.helpers.rest_client.auth import BearerTokenAuth
from dlt.sources.helpers.rest_client.paginators import HeaderLinkPaginator , JSONLinkPaginator

BASE_URL = "https://pokeapi.co/api/v2/pokemon?limit=10"

@dlt.resource(name = "pokemon" ,write_disposition="replace")
def paginated_gatter():
    client = RESTClient(
        base_url= BASE_URL,
        paginator=JSONLinkPaginator(next_url_path="next"),
        data_selector="results",
    )


    for page in client.paginate():
        for pokemon in page:
            yield pokemon
    

#Define pipeline:
pipeline = dlt.pipeline(
    pipeline_name="Pokemon_names", 
    destination="duckdb",
    dataset_name="pokemon_dataset"
)

#Run the pipeline:
load_info = pipeline.run(paginated_gatter())  # <-- No need to specify write_disposition here

df = pipeline.dataset().pokemon.df()
print(df.head())


         name                                   url        _dlt_load_id  \
0   bulbasaur  https://pokeapi.co/api/v2/pokemon/1/  1756452212.1763272   
1     ivysaur  https://pokeapi.co/api/v2/pokemon/2/  1756452212.1763272   
2    venusaur  https://pokeapi.co/api/v2/pokemon/3/  1756452212.1763272   
3  charmander  https://pokeapi.co/api/v2/pokemon/4/  1756452212.1763272   
4  charmeleon  https://pokeapi.co/api/v2/pokemon/5/  1756452212.1763272   

          _dlt_id  
0  YlgiDfj4OxHMdA  
1  qvbBjbyoyq0Aww  
2  twJE38P8pKAK6Q  
3  uFCcg6nmNkGhGw  
4  NDtMVL/6wRdFag  
